### graph on montly price of  1-3 Month U.S. Treasury Bills (T-Bills) and the interest rates in U.S.

In [4]:
import pandas as pd
import plotly.express as px
from fredapi import Fred
import os
from dotenv import load_dotenv
import json

In [ ]:
load_dotenv()

# Replace with your actual FRED API Key
API_KEY = os.getenv('FRED_API_KEY')
fred = Fred(api_key=API_KEY)

# Fetch data from FRED
start_date = "2019-03-01"  # Start 5 years ago
end_date = "2024-03-01"    # End date

# Get 3-Month Treasury Bill Yield (Secondary Market Rate)
t_bill = fred.get_series("TB3MS", start_date, end_date)

# German 3-Month Schatz (Bundesobligationen)
bundesobligationen = fred.get_series("IRLTLT01DEM156N", start_date, end_date)

# Get Federal Funds Rate
fed_funds = fred.get_series("FEDFUNDS", start_date, end_date)

# Convert to DataFrame and resample monthly
df = pd.DataFrame({"Date": t_bill.index, "3M T-Bill Yield": t_bill, "Fed Funds Rate": fed_funds, "Bundesobligationen": bundesobligationen})
df = df.resample("ME", on="Date").mean().reset_index()  # Ensure monthly data

# Create interactive plot using Plotly
fig = px.line(df, x="Date", y=["3M T-Bill Yield", "Fed Funds Rate", "Bundesobligationen"],
              labels={"value": "Interest Rate (%)", "variable": "Rate Type"},
              title="1-3 Month U.S. Treasury Bill Yields vs. Federal Funds Rate vs Bundesobligationen (Last 5 Years)")

# Customize layout
fig.update_layout(xaxis_title="Year", yaxis_title="Interest Rate (%)", 
                  legend_title="Rate Type", template="plotly_dark")

# Show plot
fig.show()


In [4]:
df

,Date,3M T-Bill Yield,Fed Funds Rate,Bundesobligationen
0,2019-03-31,2.40,2.41,0.010952
1,2019-04-30,2.38,2.42,-0.039500
2,2019-05-31,2.35,2.39,-0.134545
3,2019-06-30,2.17,2.38,-0.315263
4,2019-07-31,2.10,2.40,-0.388261
...,...,...,...,...
56,2023-11-30,5.27,5.33,2.601364
57,2023-12-31,5.24,5.33,2.102105
58,2024-01-31,5.22,5.33,2.174545
59,2024-02-29,5.24,5.33,2.331905


### Tbill absolute prices

In [23]:
import yfinance as yf

# Define the ETF tickers (adjust based on your exchange)
etf_tickers = {
    "SPDR 1-3M T-Bill ETF": "ZPR1.DE",  # IE00BJXRT698 (Xetra)
    "iShares US Treasury Bond 0-1yr": "AGGU.L" # "IB01.DE"-delisted  # IE00BZ043R46 (Xetra)
}

# Fetch historical price data (5 years)
etf_data = {}
for name, ticker in etf_tickers.items():
    etf = yf.Ticker(ticker)
    df = etf.history(period="5y", interval="1d")["Close"].rename(name)  # Get close price
    df.index = df.index.tz_convert('UTC').tz_localize(None)
    df.index = df.index.normalize()

    etf_data[name] = df

# Combine data into a single DataFrame
df = pd.concat(etf_data, axis=1).reset_index()
# df.dropna(how='any', axis=0, inplace=True)

# Melt the DataFrame for Plotly
df_melted = df.melt(id_vars=["Date"], var_name="ETF", value_name="Price")

# Create interactive Plotly line chart
fig = px.line(df_melted, x="Date", y="Price", color="ETF",
              title="Price Evolution of SPDR 1-3M T-Bill ETF & iShares US Treasury Bond ETF (Last 5 Years)",
              labels={"Price": "ETF Price (EUR)", "Date": "Year"},
              template="plotly_dark")

# Customize layout
fig.update_layout(xaxis_title="Date", yaxis_title="ETF Price",
                  hovermode="x unified", legend_title="ETF Name")

# Show plot
fig.show()


In [24]:
df

,Date,SPDR 1-3M T-Bill ETF,iShares US Treasury Bond 0-1yr
0,2020-03-09,NaN,5.7355
1,2020-03-10,NaN,5.6885
2,2020-03-11,NaN,5.6630
3,2020-03-12,NaN,5.5200
4,2020-03-13,NaN,5.5650
...,...,...,...
1301,2025-03-03,114.150002,5.6100
1302,2025-03-04,114.055000,5.6270
1303,2025-03-05,114.125000,5.5910
1304,2025-03-06,114.294998,5.5620
